In [ ]:
import importlib
import datetime
import random

import folium
import haversine
import pandas as pd
import numpy as np
from geopy import distance as gp_distance
import matplotlib.pyplot as plt

from projects.common import geo
from projects.common.nile import dates, test_utils
from projects.cpo_model import project_config, default_values, common

In [ ]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 100

In [ ]:
_ = importlib.reload(default_values)
_ = importlib.reload(project_config)
_ = importlib.reload(common)
_ = importlib.reload(test_utils)
_ = importlib.reload(geo)

In [ ]:
cluster = project_config.get_project_cluster(
    parallel_operations_limit=5,
    pool='taxi-delivery',
    extra_requirements=[],
)

### Assumptions

#### Модель
- Модель описывает только К2 на последней миле, все остальное не учитывается
- Все расстояния считаются по прямой, но скорость курьера - средняя скорость такого типа курьера в сервисе от двери до двери (включая подъем на этаж и тд, т.е. целиком)
- Мы знаем получасовой слот времени доставки для всех доставок (и Лавки и Маркета)
- В нашем мире бесконечный саплай курьеров и невероятная эффективность: каждый раз, когда нужен курьер - он стоит прямо у ПВЗ/Лавки
- Предполагаем, что ПВЗ - то же самое, что лавка (в той же локации), только без заказов лавки
- Место в ПВЗ/Лавках никогда не кончается
- Время доставки для юзера считается как случайное время во временном интервале, который он выбрал

#### Продукт
- Приносим только заказы до 10кг, но объем не учитываем
- SLA доставки (для маркета и лавки одинаковый) - 10 минут
- Если курьер на велосипеде физически не может успеть отвезти посылку за 10 минут от ПВЗ/Лавки - мы не даем возможность заказать такую доставку


#### Supply
- Все курьеры - велосипедисты
- Все курьеры получают деньги за SH - 260р в Мск и 250р в Казани

- Курьер может унести не больше 10кг за раз, может взять не больше 3-х посылок, может быть в пути не больше 4-х часов

#### Батчинг
- Принцип батчинга: если путь от точки Б первой доставки до точки Б второй < SLA и попадает в слот от юзера, можно забатчить такие посылки
- Мы не батчим с другими заказами доставки

### Define functions

In [ ]:
def convert_to_datetime(dttm_str):
    return datetime.datetime.strptime(dttm_str, '%Y-%m-%d %H:%M:%S')

def choose_random_time_in_interval(dttm_left, dttm_right):
    delta = (dttm_right - dttm_left).total_seconds()
    return dttm_left + datetime.timedelta(seconds=random.randint(0, delta))

def parse_gps_str(gps_str):
    return tuple(float(c) for c in gps_str.split(','))

def assign_day(dttm, month, day):
    return dttm.replace(month=month, day=day)

### Get and prepare dataframes

In [ ]:
# lavka_places_df = test_utils.decode_dataframe(
#     cluster
#     .read('//home/taxi-delivery/analytics/dev/cpo_model/lavka_places_moscow')
#     .as_dataframe()
# )
# lavka_places_df = lavka_places_df[pd.isnull(lavka_places_df['disable_details_status'])].reset_index()

In [ ]:
eda_places_df = test_utils.decode_dataframe(
    cluster
    .read(default_values.YT.EDA_PLACES)
    .as_dataframe()
)

In [ ]:
eda_places_df.head()

In [ ]:
lavka_places_df = eda_places_df[
    (eda_places_df['enabled']==1)
    & (eda_places_df['name']=='Яндекс.Лавка')
    & (eda_places_df['location_lat'] < 56.1)
    & (eda_places_df['location_lat'] > 55)
    & (eda_places_df['location_lon'] < 38.3)
    & (eda_places_df['location_lon'] > 37)
][[
    'location_lat', 'location_lon', 'name', 'address_city',
    'address_full', 'couriers_type', 'disable_details_status',
    'utc_disable_details_available_dttm'
]]

In [ ]:
lavka_places_df.head()

In [ ]:
additional_pp = pd.read_excel('./inputs/pp_list_2021-03-18.xlsx')

In [ ]:
additional_pp.head()

In [ ]:
additional_pp = additional_pp[additional_pp['city'].isin({'Москва', 'Московская область'})]

In [ ]:
# lavka_orders_df = test_utils.decode_dataframe(
#     cluster
#     .read('//home/taxi-delivery/analytics/dev/cpo_model/lavka_orders_moscow_2020-10-29_2020-11-04')
#     .as_dataframe()
# )

In [ ]:
# lavka_orders_df.head()

In [ ]:
market_orders_df = test_utils.decode_dataframe(
    cluster
    .read('//home/taxi-delivery/analytics/dev/cpo_model/moscow_deliveries_2020')
    .as_dataframe()
)
market_orders_df = market_orders_df[~pd.isnull(market_orders_df['final_delivery_point_gps'])]

In [ ]:
market_orders_df.head(2)

In [ ]:
# Pickup Points.
pickup_points_df = pd.DataFrame()
pickup_points_df['pickup_point_id'] = np.arange(len(lavka_places_df))
pickup_points_df['lat'] = lavka_places_df['location_lat'].values
pickup_points_df['lon'] = lavka_places_df['location_lon'].values
pickup_points_df['take_start_tm'] = '06:00:00'
pickup_points_df['take_end_tm'] = '07:00:00'
pickup_points_df['address'] = lavka_places_df['address_full'].values
pickup_points_df['capacity'] = 150
pickup_points_df['pp_type'] = 'lavka'
pickup_points_df['sla_sec'] = 30*60
pickup_points_df['delivery_product_type'] = 'slot'

In [ ]:
add_pickup_points_df = pd.DataFrame()
add_pickup_points_df['pickup_point_id'] = 1000 + np.arange(len(additional_pp))
add_pickup_points_df['lat'] = additional_pp['lat'].values
add_pickup_points_df['lon'] = additional_pp['lon'].values
add_pickup_points_df['take_start_tm'] = '06:00:00'
add_pickup_points_df['take_end_tm'] = '07:00:00'
add_pickup_points_df['address'] = additional_pp['address'].values
add_pickup_points_df['capacity'] = additional_pp['capacity'].values
add_pickup_points_df['pp_type'] = 'pvz'
add_pickup_points_df['sla_sec'] = 60*60
add_pickup_points_df['delivery_product_type'] = 'slot'

In [ ]:
pickup_points_df = pd.concat([pickup_points_df, add_pickup_points_df], axis=0)

In [ ]:
pickup_points_df.head()

In [ ]:
# # Lavka Orders.
# lavka_deliveries_df = pd.DataFrame()
# lavka_deliveries_df['service_name'] = ['lavka']*len(lavka_orders_df)
# lavka_deliveries_df['weight'] = 2
# lavka_deliveries_df['volume'] = None
# lavka_deliveries_df['delivery_lat'] = lavka_orders_df['location_lat']
# lavka_deliveries_df['delivery_lon'] = lavka_orders_df['location_lon']
# lavka_deliveries_df['warehouse_lat'] = None
# lavka_deliveries_df['warehouse_lon'] = None
# lavka_deliveries_df['warehouse_id'] = None
# lavka_deliveries_df['delivery_plan_dttm'] = lavka_orders_df['local_created_dttm'].apply(
#     lambda x: assign_day(convert_to_datetime(x) + datetime.timedelta(minutes=30), 10, 29)
# )
# lavka_deliveries_df['warehouse_ready_dttm'] = None

In [ ]:
# lavka_deliveries_df.head()

In [ ]:
market_orders_df = market_orders_df[market_orders_df['delivery_type']=='Курьерская доставка']
market_orders_df = market_orders_df[market_orders_df['order_physical_weight']<=30]

In [ ]:
market_orders_df['original_delivery_from_datetime'] = (
    market_orders_df['original_delivery_from_datetime'].astype(str).apply(
        lambda x: x[:10] + ' ' + '09:00:00' if x[11:] < '09:00:00' else x
    ).apply(
        lambda x: assign_day(convert_to_datetime(x), 11, 2)
    )
)
market_orders_df['original_delivery_to_datetime'] = (
    market_orders_df['original_delivery_to_datetime'].astype(str).apply(
        lambda x: x[:10] + ' ' + '23:00:00' if x[11:] > '23:00:00' else x
    ).apply(
        lambda x: assign_day(convert_to_datetime(x), 11, 2)
    )
)

In [ ]:
# Market Orders.
market_deliveries_df = pd.DataFrame()
market_deliveries_df['service_name'] = ['market']*len(market_orders_df)
market_deliveries_df['weight'] = market_orders_df['order_physical_weight'].fillna(
    np.mean(market_orders_df['order_physical_weight'])
)
market_deliveries_df['volume'] = market_orders_df['order_volume']*1000000
market_deliveries_df['delivery_lat'] = (
    market_orders_df['final_delivery_point_gps'].apply(lambda x: parse_gps_str(x)[1])
)
market_deliveries_df['delivery_lon'] = (
    market_orders_df['final_delivery_point_gps'].apply(lambda x: parse_gps_str(x)[0])
)
market_deliveries_df['warehouse_lat'] = None
market_deliveries_df['warehouse_lon'] = None
market_deliveries_df['warehouse_id'] = None
market_deliveries_df['delivery_plan_dttm'] = market_orders_df.apply(
    lambda x: choose_random_time_in_interval(
        x['original_delivery_from_datetime'],
        x['original_delivery_to_datetime']
    ),
    axis=1
)
market_deliveries_df['order_id_source'] = market_orders_df['order_id'].astype(int)
market_deliveries_df['warehouse_ready_dttm'] = None
market_deliveries_df = market_deliveries_df[~pd.isnull(market_deliveries_df['delivery_lat'])]
market_deliveries_df = market_deliveries_df[~pd.isnull(market_deliveries_df['delivery_lon'])]

In [ ]:
market_deliveries_df.head()

### Run simulation

In [ ]:
# simulation_paramaters = {
#     'Market_x0.5': {'frac_lavka_orders': 0., 'n_market_orders': int(17200/2)},
#     'Market_x0.5': {'frac_lavka_orders': 0., 'n_market_orders': int(17200/2)},
#     'Market_x1': {'frac_lavka_orders': 0., 'n_market_orders': int(17200)},
#     'Market_x2': {'frac_lavka_orders': 0., 'n_market_orders': int(17200*2)},
#     'Market_x3': {'frac_lavka_orders': 0., 'n_market_orders': int(17200*3)},
#     'Market_x4': {'frac_lavka_orders': 0., 'n_market_orders': int(17200*4)},
# }
simulation_paramaters = {
    '13K_orders_sla_30': {'frac_lavka_orders': 0., 'n_market_orders': int(13000), 'sla': 30},
    '13K_orders_sla_60': {'frac_lavka_orders': 0., 'n_market_orders': int(13000), 'sla': 60},
    '13K_orders_sla_120': {'frac_lavka_orders': 0., 'n_market_orders': int(13000), 'sla': 120},
    '13K_orders_sla_180': {'frac_lavka_orders': 0., 'n_market_orders': int(13000), 'sla': 180},
    '31K_orders_sla_30': {'frac_lavka_orders': 0., 'n_market_orders': int(31000), 'sla': 30},
    '31K_orders_sla_60': {'frac_lavka_orders': 0., 'n_market_orders': int(31000), 'sla': 60},
    '31K_orders_sla_120': {'frac_lavka_orders': 0., 'n_market_orders': int(31000), 'sla': 120},
    '31K_orders_sla_180': {'frac_lavka_orders': 0., 'n_market_orders': int(31000), 'sla': 180},
}

In [ ]:
simulation_paramaters

In [ ]:
# pickup_points_df.loc[
#         pickup_points_df['pp_type']=='lavka', 'sla_sec'
#     ] = 40*60

In [ ]:
# pickup_points_df

In [ ]:
simulation_dict = {}
for key, value in simulation_paramaters.items():
    simulation_dict[key] = {}
    
    deliveries_df = pd.concat(
        [
         market_deliveries_df.sample(n=value['n_market_orders'], random_state=7)],
        axis=0
    )
    print(key)
    print(deliveries_df.shape)
    
    sim = common.Simulation(pp_work_time=14)
    simulation_dict[key]['simulation'] = sim
    pickup_points_df.loc[
        pickup_points_df['pp_type']=='lavka', 'sla_sec'
    ] = value['sla']*60
    
    sim.initialize_simulation(deliveries_df, pickup_points_df)
    sim.assign_objects()
    sim.batch_deliveries_pp_to_user()
    sim.pay_couriers()
    print('\n')

In [ ]:
stats = {}
for key in simulation_dict.keys():
    
    sim = simulation_dict[key]['simulation']
    pp_lavka = [pp for pp in sim.pickup_points if pp.pp_type=='lavka']
    pp_add = [pp for pp in sim.pickup_points if pp.pp_type=='pvz']
    deliveries_lavka = [d for d in sim.deliveries if d.potential_pickup_point_id
                        in set([pp.id for pp in pp_lavka])]
    deliveries_add = [d for d in sim.deliveries if d.potential_pickup_point_id
                      in set([pp.id for pp in pp_add])]
    
    stats[(key, 'lavka')] = sim.calculate_metrics(deliveries_lavka, pp_lavka)
    stats[(key, 'pvz')] = sim.calculate_metrics(deliveries_add, pp_add)
    stats[(key, 'all')] = sim.calculate_metrics(sim.deliveries, sim.pickup_points)

In [ ]:
pd.DataFrame(stats).T#.to_excel('./outputs/k2_market_13k_31k_sla_30-180.xlsx')

In [ ]:
pd.DataFrame(stats).T

In [ ]:
# [pp.sla_sec for pp in sim.pickup_points]

In [ ]:
# pickup_points_df

In [ ]:
# pd.DataFrame(stats).T.to_excel('./outputs/k2_market_may_november.xlsx')

In [ ]:
for key, value in simulation_dict.items():

    sim = value['simulation']
    pp_list = []

    for pp in sim.pickup_points:
        pp_deliveries = [
            d for d in sim.deliveries if d.pickup_point_id==pp.id and d.delivered_flg
        ]
        pp_couriers = set([sim.couriers_dict[d.courier_id] for d in pp_deliveries])
        pp_item = {}

        pp_item['type'] = pp.pp_type
        pp_item['address'] = pp.address
        pp_item['capacity'] = pp.capacity
        pp_item['potential_deliveries'] = pp.potential_deliveries
        pp_item['stored_deliveries'] = pp.stored_deliveries

        pp_item['success_deliveries'] = len(pp_deliveries)
        pp_item['success_deliveries_bicycle'] = len(
            [d for d in pp_deliveries if sim.couriers_dict[d.courier_id].type=='bicycle']
        )
        pp_item['success_deliveries_vehicle'] = len(
            [d for d in pp_deliveries if sim.couriers_dict[d.courier_id].type=='vehicle']
        )
        pp_item['failed_deliveries_due_to_capacity'] = (
            pp_item['potential_deliveries'] - pp_item['stored_deliveries']
        )
        pp_item['failed_deliveries_due_to_large_radius'] = (
            pp_item['stored_deliveries'] - pp_item['success_deliveries']
        )
        pp_item['fraction_of_batched_deliveries'] = len(
            [d for d in pp_deliveries if d.pp_to_user_batched_flg]
        ) / (pp_item['success_deliveries'] + 1e-6)

        pp_item['total_sh'] = (
            sum([c.time_travelled * common.COURIER_HALT_COEF 
                  for c in pp_couriers])
            / 3600
        )
        pp_item['total_sh_bicycle'] = (
            sum([c.time_travelled * common.COURIER_HALT_COEF 
                  for c in pp_couriers if c.type=='bicycle'])
            / 3600
        )
        pp_item['total_sh_vehicle'] = (
            sum([c.time_travelled * common.COURIER_HALT_COEF 
                  for c in pp_couriers if c.type=='vehicle'])
            / 3600
        )
        pp_item['total_sh_cost'] = (
            pp_item['total_sh_vehicle'] * common.COURIER_SH_COST['vehicle']
            + pp_item['total_sh_bicycle'] * common.COURIER_SH_COST['bicycle']
        )
        pp_item['total_sh_cost_bicycle'] = (
            pp_item['total_sh_bicycle'] * common.COURIER_SH_COST['bicycle']
        )
        pp_item['total_sh_cost_vehicle'] = (
            pp_item['total_sh_vehicle'] * common.COURIER_SH_COST['vehicle']
        )
        pp_item['delivery_cost_sh_based'] = (
            pp_item['total_sh_cost']
            / (pp_item['success_deliveries'] + 1e-6)
        )
        pp_item['delivery_cost_sh_based_bicycle'] = (
            pp_item['total_sh_cost_bicycle']
            / (pp_item['success_deliveries_bicycle'] + 1e-6)
        )
        pp_item['delivery_cost_sh_based_vehicle'] = (
            pp_item['total_sh_cost_vehicle']
            / (pp_item['success_deliveries_vehicle'] + 1e-6)
        )
        pp_list.append(pp_item)
        
        pp_df = pd.DataFrame(pp_list)
        
        pp_df.to_excel('./outputs/k2_market_pickup_point_breakdown_{}.xlsx'.format(key))

In [ ]:
# pp_index_set = set([
#     8, 1164
# ])
# pp_id_set = set([sim.pickup_points[i].id for i in pp_index_set])
pp_id_set = set([8, 1164])

In [ ]:
[pp.id for pp in sim.pickup_points if pp.address=='Россия, Москва, Ленинский проспект, 54'
 or pp.address=='Россия, Москва, Профсоюзная улица, 103А']

In [ ]:
sim = simulation_dict['november']['simulation']

In [ ]:
d.pp_to_user_start_dttm

In [ ]:
d_item_list = []
for d in sim.deliveries:
    if d.delivered_flg and d.pickup_point_id in pp_id_set:
        d_item = {}
        d_item['pp_address'] = sim.pickup_points_dict[d.pickup_point_id].address
        d_item['delivery_coordinates'] = (d.location_lat, d.location_lon)
        d_item['delivery_plan_dttm'] = d.delivery_plan_dttm
        d_item['dist_from_pp_meters'] = haversine.haversine(
            (d.location_lat, d.location_lon),
            (sim.pickup_points_dict[d.pickup_point_id].lat,
             sim.pickup_points_dict[d.pickup_point_id].lon)
        ) * 1000
        d_item['courier_id'] = d.courier_id
        d_item['weight'] = d.weight
        d_item['volume'] = d.volume
        d_item['trip_start_dttm'] = d.pp_to_user_start_dttm
        d_item['batched_flg'] = d.pp_to_user_batched_flg

        d_item_list.append(d_item)

In [ ]:
deliveries_to_check = pd.DataFrame(d_item_list)
deliveries_to_check['hour'] = deliveries_to_check['delivery_plan_dttm'].apply(
    lambda x: datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S').hour
)

In [ ]:
deliveries_to_check['pp_address'].unique()

In [ ]:
deliveries_to_check

In [ ]:
batching_dist_list = []
for pp_address in deliveries_to_check['pp_address'].unique():
    mean_dist = deliveries_to_check[
        deliveries_to_check['pp_address']==pp_address
    ]['dist_from_pp_meters'].mean()
    batching = round(pp_df[pp_df['address']==pp_address]['fraction_of_batched_deliveries'].iloc[0], 3)
    batching_dist_list.append({'pp_address': pp_address, 'batching': batching, 'mean_dist': mean_dist})
pd.DataFrame(batching_dist_list)

In [ ]:
# fig, axs = plt.subplots(len(deliveries_to_check['pp_address'].unique()), figsize=(5, 25))
fig, axs = plt.subplots(3, 3, figsize=(20, 14))

h = -1
for n, pp_address in enumerate(deliveries_to_check['pp_address'].unique()):
    if n%3 == 0:
        h += 1
    hist = deliveries_to_check[
        deliveries_to_check['pp_address']==pp_address
    ]['hour'].hist(bins=14, ax=axs[h, n%3])
    axs[h, n%3].set_title(','.join(pp_address.split(',')[2:]))
    axs[h, n%3].legend(
        [str(round(pp_df[pp_df['address']==pp_address]['fraction_of_batched_deliveries'].iloc[0], 3))],
    )

In [ ]:
# deliveries_to_check.to_excel('./outputs/k2_deliveries_to_check.xlsx')

In [ ]:
sim = simulation_dict['may']['simulation']

In [ ]:
d_item_list = []
for d in sim.deliveries:
    d_item = {}
    
    d_item['delivery_plan_dttm'] = d.delivery_plan_dttm
    d_item_list.append(d_item)

In [ ]:
deliveries_to_check = pd.DataFrame(d_item_list)
deliveries_to_check['hour'] = deliveries_to_check['delivery_plan_dttm'].apply(
    lambda x: datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S').hour
)

In [ ]:
deliveries_to_check['hour'].hist(bins=14)

In [ ]:
market_deliveries_df[
    (market_deliveries_df['delivery_plan_dttm']>='2020-11-01 00:00:00')
    & (market_deliveries_df['delivery_plan_dttm']<'2020-12-01 00:00:00')
].shape

In [ ]:
market_deliveries_df['delivery_plan_dttm']

In [ ]:
deliveries_to_check.to_excel('./outputs/deliveries_detailed_lavka_pvz_november.xlsx')

### Optimal geo of pp

In [ ]:
import math
from nltk.cluster.kmeans import KMeansClusterer
from sklearn.metrics import pairwise_distances
import haversine

In [ ]:
radius = 10 * 132.1

deliveries = [d for d in sim.deliveries if not d.delivered_flg]
X = np.array(
    [
        [delivery.location_lat, delivery.location_lon]
        for delivery in deliveries
    ],
)

# Choose number of clusters.
num_clusters = math.ceil(len(deliveries) / 120)

success_deliveries_frac = 0
# Optimize pickup points.
while success_deliveries_frac < 0.7:
    kclusterer = KMeansClusterer(num_clusters,
                                 distance=haversine.haversine,
                                 repeats=1,
                                 avoid_empty_clusters=True,)
    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    failed_deliveries_cnt = 0
    ac_points_in_radius = {}
    for ac, d in zip(assigned_clusters, deliveries):
        dist_ac = haversine.haversine(
            kclusterer.means()[ac], (d.location_lat, d.location_lon)
        ) * 1000.
        if dist_ac > radius:
            failed_deliveries_cnt += 1

    success_deliveries_frac = (len(deliveries_df) - failed_deliveries_cnt) / len(deliveries_df)
    if success_deliveries_frac < 0.7:
        num_clusters += 1
        print('increased num clusters to {}'.format(num_clusters))

# Get pp df.
pp_addresses = pd.DataFrame(
    {
        'pp_id': list(range(len(kclusterer.means()))),
        'lat': [c[0] for c in kclusterer.means()],
        'lon': [c[1] for c in kclusterer.means()],
    }
)
# indexes = pairwise_distances(
#     pp_addresses[['lat', 'lon']].values,
#     real_kazan_pp_df[['lat', 'lon']].values,
#     metric=haversine.haversine,
#     force_all_finite=True
# ).argmin(axis=1)
# pp_addresses_real = real_kazan_pp_df.iloc[indexes]


# pickup_points_df = pd.DataFrame()
# pickup_points_df['pickup_point_id'] = 1000 + pp_addresses_real['real_pp_id']
# pickup_points_df['lat'] = pp_addresses_real['lat']
# pickup_points_df['lon'] = pp_addresses_real['lon']
# pickup_points_df['take_start_tm'] = '06:00:00'
# pickup_points_df['take_end_tm'] = '07:00:00'
# pickup_points_df['address'] = None

# Run simulation.
# sim.initialize_simulation(deliveries_df, pickup_points_df)
# sim.assign_objects()
# sim.batch_deliveries_pp_to_user()
# sim.pay_couriers()
# stats[(key, sla, 'all')] = sim.calculate_metrics(sim.deliveries)
# print('\n')

In [ ]:
success_deliveries_frac

In [ ]:
real_pp_df = pd.read_excel('./import_data/pp_list.xlsx', skiprows=3)
real_pp_df = real_pp_df[(real_pp_df['Населённый пункт']=='г Москва')
& (real_pp_df['Тип пункта']=='ПВЗ')][['Широта', 'Долгота']].drop_duplicates(subset=('Широта', 'Долгота')).reset_index(
).rename(
    {'index': 'real_pp_id', 'Широта': 'lat', 'Долгота': 'lon'}, axis=1
)

In [ ]:
pp_addresses.to_excel('./pp_addresses_moscow_estimate.xlsx')

### Draw orders on map

In [ ]:
sim = simulation_dict['november']['simulation']

In [ ]:
YA_MAPS_TILES = 'http://vec02.maps.yandex.net/tiles?l=map&v=2.2.3&x={x}&y={y}&z={z}'

In [ ]:
f_map = folium.Map(
    location=(55.7, 37.5), zoom_start=10, tiles=YA_MAPS_TILES, max_zoom=20,
    attr='My Data Attribution', crs='EPSG3395'
)

In [ ]:
# for d in sim.deliveries:
#     c = 'red'
#     if d.delivered_flg:
#         if d.pickup_point_id >= 1000:
#             c = 'green'
#         else:
#             c = 'blue'
#     folium.CircleMarker(
#         location=(d.location_lat, d.location_lon),
#         radius=1.,
#         color=c,
#         fill=True,
#         weight=1.,
#     ).add_to(f_map)
    
for pp in sim.pickup_points:
    if pp.pp_type == 'pvz':
        c = 'green'
    if pp.pp_type == 'lavka':
        c = 'blue'
    folium.CircleMarker(
        location=(pp.lat, pp.lon),
        radius=3,
        color=c,
        fill=True,
        weight=3,
    ).add_to(f_map)
    
# for row in real_pp_df.itertuples():
#     c = 'orange'
#     folium.CircleMarker(
#         location=(row.lat, row.lon),
#         radius=3,
#         color=c,
#         fill=True,
#         weight=3,
#     ).add_to(f_map)

In [ ]:
f_map.save('./outputs/maps/moscow_market_november_deliveries.html')

In [ ]:
f_map.save('./outputs/maps/moscow_market_pickup_points.html')

### Get k2 vs ndd data

In [ ]:
market_deliveries_df['weight'].mean() * 41

In [ ]:
market_orders_df['order_volume'].mean() * 41

In [ ]:
sim = simulation_dict['Market_x1']['simulation']

In [ ]:
for pp in sim.pickup_points:
    pp.deliveries = [d for d in sim.deliveries if d.pickup_point_id == pp.id]

In [ ]:
for pp in sim.pickup_points:
    print(pp.id, len([d for d in pp.deliveries if d.delivered_flg]))

In [ ]:
pp = sim.pickup_points_dict[16]

In [ ]:
oid_set = set([d.order_id_source for d in pp.deliveries if d.delivered_flg])

In [ ]:
orders = market_orders_df[
    market_orders_df['order_id'].isin(oid_set)
][['order_id', 'final_delivery_point_address',
   'original_delivery_from_datetime', 'original_delivery_to_datetime',]]

In [ ]:
orders_excel = pd.DataFrame()
orders_excel['id'] = orders['order_id']
orders_excel['address'] = orders['final_delivery_point_address']
orders_excel['time_window'] = orders.apply(
    lambda x: ' - '.join((str(x['original_delivery_from_datetime'])[11:16],
                          str(x['original_delivery_to_datetime'])[11:16])), axis=1
)
orders_excel['shared_service_duration_s'] = 240
orders_excel['service_duration_s'] = 120

In [ ]:
orders_excel.to_excel('./analytics/k2_vs_ndd_orders_moscow.xlsx')

In [ ]:
market_orders_df[
    market_orders_df['order_id'].isin(oid_set)
]

In [ ]:
for obj in sorted(
    sim.pickup_points,
    key=(lambda x: haversine.haversine((x.lat, x.lon), (pp.lat, pp.lon)))
):
    print(obj.id, len([d for d in obj.deliveries if d.delivered_flg]))

In [ ]:
print(sim.pickup_points_dict[16].lat, sim.pickup_points_dict[16].lon)
print(sim.pickup_points_dict[107].lat, sim.pickup_points_dict[107].lon)
print(sim.pickup_points_dict[47].lat, sim.pickup_points_dict[47].lon)
print(sim.pickup_points_dict[92].lat, sim.pickup_points_dict[92].lon)
print(sim.pickup_points_dict[11].lat, sim.pickup_points_dict[11].lon)
print(sim.pickup_points_dict[104].lat, sim.pickup_points_dict[104].lon)

### Get random orders for yulia

In [ ]:
import random

In [ ]:
rand_deliveries = (
    random.sample(sim.pickup_points[0].deliveries, 10)
    + random.sample(sim.pickup_points[2].deliveries, 10)
    + random.sample(sim.pickup_points[6].deliveries, 10)
    + random.sample(sim.pickup_points[12].deliveries, 10)
)

In [ ]:
oid_set = set([d.order_id_source for d in rand_deliveries])

In [ ]:
orders = market_orders_df[
    market_orders_df['order_id'].isin(oid_set)
][['order_id', 'final_delivery_point_address',
   'original_delivery_from_datetime', 'original_delivery_to_datetime',]]

In [ ]:
orders_excel = pd.DataFrame()
orders_excel['id'] = orders['order_id']
orders_excel['address'] = orders['final_delivery_point_address']
orders_excel['time_window'] = orders.apply(
    lambda x: ' - '.join((str(x['original_delivery_from_datetime'])[11:16],
                          str(x['original_delivery_to_datetime'])[11:16])), axis=1
)
orders_excel['shared_service_duration_s'] = 240
orders_excel['service_duration_s'] = 120

In [ ]:
orders_excel.to_excel('./analytics/k2_vs_ndd_orders_strig_pricing.xlsx')

In [ ]:
collections.Counter([c.deliveries_made for c in  sim.couriers])